In [1]:
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
import numpy as np
from scipy import sparse
from tqdm.notebook import tqdm

In [3]:
from p_tqdm import p_map, p_umap

In [4]:
!ls ../data/processed

A_tr.npz   B_tr.npz  meta_tr.csv   P_tr.npz
A_tst.npz  meta.csv  meta_tst.csv  test.cor


In [5]:
A_tr = sparse.load_npz('../data/processed/A_tr.npz')
B_tr = sparse.load_npz('../data/processed/B_tr.npz')
P_tr = sparse.load_npz('../data/processed/P_tr.npz')
A_tr_csr = A_tr
A_tr_csc = A_tr.tocsc(copy=True)  # memory is cheap ;D

In [6]:
A_tr

<656x1563400 sparse matrix of type '<class 'numpy.int64'>'
	with 7570310 stored elements in Compressed Sparse Row format>

In [10]:
meta_tr = pd.read_csv('../data/processed/meta_tr.csv', index_col=0)

In [34]:
meta_tr['counts'] = np.asarray(A_tr.sum(axis=1)).T[0]

In [35]:
meta_tr.groupby('label').counts.describe()

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
class0,328.0,19980.384146,11644.184581,61.0,11712.0,19369.5,27750.75,52508.0
class1,328.0,3099.829268,4920.820259,87.0,276.0,648.0,4849.00,44350.0


In [7]:
import networkx as nx

In [8]:
A_tr

<656x1563400 sparse matrix of type '<class 'numpy.int64'>'
	with 7570310 stored elements in Compressed Sparse Row format>

In [9]:
B_tr

<1563400x1563400 sparse matrix of type '<class 'numpy.int8'>'
	with 38980208 stored elements in Compressed Sparse Column format>

In [11]:
P_tr

<1563400x1563400 sparse matrix of type '<class 'numpy.int8'>'
	with 27097178 stored elements in Compressed Sparse Column format>

In [ ]:
A_tr_edges = []
for i, row in tqdm(enumerate(A_tr), total=A_tr.shape[0]):
    for j in row.indices:
        A_tr_edges.append([f'app_{i}', f'api_{j}'])

df_A = pd.DataFrame(A_tr_edges, columns=['source', 'target'])

In [ ]:
B_tr_edges = []
for i, row in tqdm(enumerate(B_tr), total=B_tr.shape[1]):
    for j in row.indices:
        B_tr_edges.append([f'api_{i}', f'api_{j}'])

df_B = pd.DataFrame(B_tr_edges, columns=['source', 'target'])

In [ ]:
P_tr_edges = []
for i, row in tqdm(enumerate(P_tr), total=P_tr.shape[1]):
    for j in row.indices:
        P_tr_edges.append([f'api_{i}', f'api_{j}'])

df_P = pd.DataFrame(P_tr_edges, columns=['source', 'target'])

In [72]:
del A_tr_edges
del B_tr_edges
del P_tr_edges

In [74]:
df_B.index = np.arange(df_A.shape[0], df_A.shape[0] + df_B.shape[0])
df_P.index = np.arange(df_A.shape[0] + df_B.shape[0], df_A.shape[0] + df_B.shape[0] + df_P.shape[0])

In [75]:
df_P.index[-1]

73647695

In [79]:
app_nodes = pd.DataFrame([], index=[f'app_{i}' for i in range(A_tr.shape[0])])
api_nodes = pd.DataFrame([], index=[f'api_{i}' for i in range(B_tr.shape[0])])

In [13]:
from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph

In [80]:
%%time
graph = StellarGraph(
    nodes={'APP': app_nodes, 'API': api_nodes},
    edges={'A': df_A, 'B': df_B, 'P': df_P},
    is_directed=False,
    dtype='int8'
)

CPU times: user 3min 30s, sys: 23.6 s, total: 3min 53s
Wall time: 3min 53s


In [83]:
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 1564056, Edges: 73647696

 Node types:
  API: [1563400]
    Features: none
    Edge types: API-A->APP, API-B->API, API-P->API
  APP: [656]
    Features: none
    Edge types: APP-A->API

 Edge types:
    API-B->API: [38980208]
    API-P->API: [27097178]
    APP-A->API: [7570310]


In [100]:
rw = BiasedRandomWalk(graph)

In [101]:
walks = rw.run(
    nodes=list(graph.nodes()),  # root nodes
    length=100,  # maximum length of a random walk
    n=2,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

KeyboardInterrupt: 

In [99]:
import numpy as np
import warnings
from collections import defaultdict, deque
from scipy import stats
from scipy.special import softmax

from stellargraph import GraphSchema
from stellargraph import StellarGraph
from stellargraph.core.utils import is_real_iterable
from stellargraph.core.experimental import experimental
from stellargraph.random import random_state


class GraphWalk(object):
    """
    Base class for exploring graphs.
    """

    def __init__(self, graph, graph_schema=None, seed=None):
        self.graph = graph

        # Initialize the random state
        self._check_seed(seed)

        self._random_state, self._np_random_state = random_state(seed)

        # We require a StellarGraph for this
        if not isinstance(graph, StellarGraph):
            raise TypeError("Graph must be a StellarGraph or StellarDiGraph.")

        if not graph_schema:
            self.graph_schema = self.graph.create_graph_schema()
        else:
            self.graph_schema = graph_schema

        if type(self.graph_schema) is not GraphSchema:
            self._raise_error(
                "The parameter graph_schema should be either None or of type GraphSchema."
            )

    def get_adjacency_types(self):
        # Allow additional info for heterogeneous graphs.
        adj = getattr(self, "adj_types", None)
        if not adj:
            # Create a dict of adjacency lists per edge type, for faster neighbour sampling from graph in SampledHeteroBFS:
            self.adj_types = adj = self.graph._adjacency_types(self.graph_schema)
        return adj

    def _check_seed(self, seed):
        if seed is not None:
            if type(seed) != int:
                self._raise_error(
                    "The random number generator seed value, seed, should be integer type or None."
                )
            if seed < 0:
                self._raise_error(
                    "The random number generator seed value, seed, should be non-negative integer or None."
                )

    def _get_random_state(self, seed):
        """
        Args:
            seed: The optional seed value for a given run.

        Returns:
            The random state as determined by the seed.
        """
        if seed is None:
            # Restore the random state
            return self._random_state
        # seed the random number generator
        rs, _ = random_state(seed)
        return rs

    def neighbors(self, node):
        if not self.graph.has_node(node):
            self._raise_error("node {} not in graph".format(node))
        return self.graph.neighbors(node)

    def run(self, *args, **kwargs):
        """
        To be overridden by subclasses. It is the main entry point for performing random walks on the given
        graph.

        It should return the sequences of nodes in each random walk.
        """
        raise NotImplementedError

    def _raise_error(self, msg):
        raise ValueError("({}) {}".format(type(self).__name__, msg))

    def _check_common_parameters(self, nodes, n, length, seed):
        """
        Checks that the parameter values are valid or raises ValueError exceptions with a message indicating the
        parameter (the first one encountered in the checks) with invalid value.

        Args:
            nodes: <list> A list of root node ids from which to commence the random walks.
            n: <int> Number of walks per node id.
            length: <int> Maximum length of each walk.
            seed: <int> Random number generator seed.
        """
        self._check_nodes(nodes)
        self._check_repetitions(n)
        self._check_length(length)
        self._check_seed(seed)

    def _check_nodes(self, nodes):
        if nodes is None:
            self._raise_error("A list of root node IDs was not provided.")
        if not is_real_iterable(nodes):
            self._raise_error("Nodes parameter should be an iterable of node IDs.")
        if (
            len(nodes) == 0
        ):  # this is not an error but maybe a warning should be printed to inform the caller
            warnings.warn(
                "No root node IDs given. An empty list will be returned as a result.",
                RuntimeWarning,
                stacklevel=3,
            )

    def _check_repetitions(self, n):
        if type(n) != int:
            self._raise_error(
                "The number of walks per root node, n, should be integer type."
            )
        if n <= 0:
            self._raise_error(
                "The number of walks per root node, n, should be a positive integer."
            )

    def _check_length(self, length):
        if type(length) != int:
            self._raise_error("The walk length, length, should be integer type.")
        if length <= 0:
            # Technically, length 0 should be okay, but by consensus is invalid.
            self._raise_error("The walk length, length, should be a positive integer.")

    # For neighbourhood sampling
    def _check_sizes(self, n_size):
        err_msg = "The neighbourhood size must be a list of non-negative integers."
        if not isinstance(n_size, list):
            self._raise_error(err_msg)
        if len(n_size) == 0:
            # Technically, length 0 should be okay, but by consensus it is invalid.
            self._raise_error("The neighbourhood size list should not be empty.")
        for d in n_size:
            if type(d) != int or d < 0:
                self._raise_error(err_msg)

                
class BiasedRandomWalk(GraphWalk):
    """
    Performs biased second order random walks (like those used in Node2Vec algorithm
    https://snap.stanford.edu/node2vec/) controlled by the values of two parameters p and q.
    """

    def run(self, nodes, n, length, p=1.0, q=1.0, seed=None, weighted=False):

        """
        Perform a random walk starting from the root nodes.

        Args:
            nodes (list): The root nodes as a list of node IDs
            n (int): Total number of random walks per root node
            length (int): Maximum length of each random walk
            p (float, default 1.0): Defines probability, 1/p, of returning to source node
            q (float, default 1.0): Defines probability, 1/q, for moving to a node away from the source node
            seed (int, optional): Random number generator seed; default is None
            weighted (bool, default False): Indicates whether the walk is unweighted or weighted

        Returns:
            List of lists of nodes ids for each of the random walks

        """
        self._check_common_parameters(nodes, n, length, seed)
        self._check_weights(p, q, weighted)
        rs = self._get_random_state(seed)

        if weighted:
            # Check that all edge weights are greater than or equal to 0.
            # Also, if the given graph is a MultiGraph, then check that there are no two edges between
            # the same two nodes with different weights.
            for node in self.graph.nodes():
                # TODO Encapsulate edge weights
                for neighbor in self.graph.neighbors(node):

                    wts = set()
                    for weight in self.graph._edge_weights(node, neighbor):
                        if weight is None or np.isnan(weight) or weight == np.inf:
                            self._raise_error(
                                "Missing or invalid edge weight ({}) between ({}) and ({}).".format(
                                    weight, node, neighbor
                                )
                            )
                        if not isinstance(weight, (int, float)):
                            self._raise_error(
                                "Edge weight between nodes ({}) and ({}) is not numeric ({}).".format(
                                    node, neighbor, weight
                                )
                            )
                        if weight < 0:  # check if edge has a negative weight
                            self._raise_error(
                                "An edge weight between nodes ({}) and ({}) is negative ({}).".format(
                                    node, neighbor, weight
                                )
                            )

                        wts.add(weight)
                    if len(wts) > 1:
                        # multigraph with different weights on edges between same pair of nodes
                        self._raise_error(
                            "({}) and ({}) have multiple edges with weights ({}). Ambiguous to choose an edge for the random walk.".format(
                                node, neighbor, list(wts)
                            )
                        )

        ip = 1.0 / p
        iq = 1.0 / q

        walks = []
        for node in tqdm(nodes):  # iterate over root nodes
            for walk_number in range(n):  # generate n walks per root node
                # the walk starts at the root
                walk = [node]

                neighbours = self.neighbors(node)

                previous_node = node
                previous_node_neighbours = neighbours

                # calculate the appropriate unnormalised transition
                # probability, given the history of the walk
                def transition_probability(nn, current_node, weighted):

                    if weighted:
                        # TODO Encapsulate edge weights
                        weight_cn = self.graph._edge_weights(current_node, nn)[0]
                    else:
                        weight_cn = 1.0

                    if nn == previous_node:  # d_tx = 0
                        return ip * weight_cn
                    elif nn in previous_node_neighbours:  # d_tx = 1
                        return 1.0 * weight_cn
                    else:  # d_tx = 2
                        return iq * weight_cn

                if neighbours:
                    current_node = rs.choice(neighbours)
                    for _ in range(length - 1):
                        walk.append(current_node)
                        neighbours = self.neighbors(current_node)

                        if not neighbours:
                            break

                        # select one of the neighbours using the
                        # appropriate transition probabilities
                        choice = naive_weighted_choices(
                            rs,
                            (
                                transition_probability(nn, current_node, weighted)
                                for nn in neighbours
                            ),
                        )

                        previous_node = current_node
                        previous_node_neighbours = neighbours
                        current_node = neighbours[choice]

                walks.append(walk)

        return walks


    def _check_weights(self, p, q, weighted):
        """
        Checks that the parameter values are valid or raises ValueError exceptions with a message indicating the
        parameter (the first one encountered in the checks) with invalid value.

        Args:
            p: <float> The backward walk 'penalty' factor.
            q: <float> The forward walk 'penalty' factor.
            weighted: <False or True> Indicates whether the walk is unweighted or weighted.
       """
        if p <= 0.0:
            self._raise_error("Parameter p should be greater than 0.")

        if q <= 0.0:
            self._raise_error("Parameter q should be greater than 0.")

        if type(weighted) != bool:
            self._raise_error(
                "Parameter weighted has to be either False (unweighted random walks) or True (weighted random walks)."
            )

def naive_weighted_choices(rs, weights):
    """
    Select an index at random, weighted by the iterator `weights` of
    arbitrary (non-negative) floats. That is, `x` will be returned
    with probability `weights[x]/sum(weights)`.

    For doing a single sample with arbitrary weights, this is much (5x
    or more) faster than numpy.random.choice, because the latter
    requires a lot of preprocessing (normalized probabilties), and
    does a lot of conversions/checks/preprocessing internally.
    """

    # divide the interval [0, sum(weights)) into len(weights)
    # subintervals [x_i, x_{i+1}), where the width x_{i+1} - x_i ==
    # weights[i]
    subinterval_ends = []
    running_total = 0
    for w in weights:
        if w < 0:
            raise ValueError("Detected negative weight: {}".format(w))
        running_total += w
        subinterval_ends.append(running_total)

    # pick a place in the overall interval
    x = rs.random() * running_total

    # find the subinterval that contains the place, by looking for the
    # first subinterval where the end is (strictly) after it
    for idx, end in enumerate(subinterval_ends):
        if x < end:
            break

    return idx

## Sample paths strictly using node2vec

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
from src.features import n2v
# del n2v
n2v = n2v.Node2Vec(A_tr, B_tr, P_tr)

In [ ]:
n2v.perform_one_walk_full()

In [ ]:
n2v.perform_one_walk_metapath()

In [ ]:
walks = n2v.save_corpus()

In [ ]:
%load_ext line_profiler

In [ ]:
!wc -l node2vec_n=1_p=2_q=1_wl=100.cor

In [ ]:
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'node2vec_n=1_p=2_q=1_wl=100.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.strip().split(' ')

In [ ]:
sentences = MyCorpus()

In [ ]:
%%time

import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, min_count=1, size=200)

In [ ]:
!ls HinDroid-with-Embeddings/data/processed/

In [ ]:
meta_tr = pd.read_csv('HinDroid-with-Embeddings/data/processed/meta_tr.csv', index_col=0)

In [ ]:
meta_tr.head()

In [ ]:
y_train = meta_tr.label == 'class1'

In [ ]:
app_vec = np.array([model.wv[f'app_{i}'] for i in range(len(meta_tr))])

In [ ]:
app_vec

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')

In [ ]:
svm.fit(app_vec, y_train)

In [ ]:
svm.score(app_vec, y_train)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        if 'api' in word: continue
        tokens.append(model.wv[word])
        labels.append(word)
    
    tsne_model = TSNE(n_components=2)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 12)) 
    for i in range(len(x)):
        color = 'b' if meta_tr.label.iloc[i] == 'class1' else 'r'
        plt.scatter(x[i],y[i],c=color)
#         plt.annotate(labels[i],
#                      xy=(x[i], y[i]),
#                      xytext=(5, 2),
#                      textcoords='offset points',
#                      ha='right',
#                      va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)